In [1]:
import subprocess as sp
import os
from datetime import datetime, timedelta
import numpy as np
import pandas as pd

In [2]:
def listall(RootFolder, varname='',extension='.png'):
    lists = [os.path.join(root, name)
             for root, dirs, files in os.walk(RootFolder)
             for name in files
             if varname in name
             if name.endswith(extension)]
    return lists

# see note below
def youCanQuoteMe(item):
    return "\"" + item + "\""   

In [15]:
files = listall(r'D:\Downloads', varname='h27v05', extension='.hdf')

In [16]:
files = [files[1]]

In [19]:
index = []
for i in files:
    # get date
    year = int(i[-36:-32])
    doy = int(i[-32:-29])
    date = datetime(year, 1, 1) + timedelta(doy - 1)
    date = np.datetime64(date)
    date = pd.Timestamp(np.datetime_as_string(date))
    index.append(date)
index = np.array(index)
df = pd.DataFrame(index=index)

In [20]:
df

""
2008-01-17 00:00:00+00:00


In [21]:
month = df.index.map(lambda x: x.month)
year = df.index.map(lambda x: x.year)
day = df.index.map(lambda x: x.day)
doy =  df.index.map(lambda x: x.dayofyear)
# select only dates between certain month
df_sel = (df[(month >= 1) & (month <=6)] + 
          df[(month >= 10) & (month <=12)]).sort_index()
index_sel = pd.Series(np.in1d(df.index,df_sel.index))

In [26]:
# prulletaria
gdal_translate = r'C:\Program Files\GDAL//gdal_translate.exe'
gdalwarp = r'C:\Program Files\GDAL//gdalwarp.exe'
out_base = r'D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//'

extent = [116.84693268,117.44,33.26,34.10324112]

for idx, val in enumerate(index_sel[0:1]):    
    if val == True:
        # 1. convert from HDF to TIF
        out_file = out_base+'MOD09A1_'+str(index[idx].year)+'-'+str(index[idx].month).zfill(2)+'-'+str(index[idx].day).zfill(2)
        src = 'HDF4_EOS:EOS_GRID:'+files[idx]+':MOD_Grid_500m_Surface_Reflectance:sur_refl_b04'
        command = [gdal_translate,'-of','GTiff', src, out_file+'.tif']
        print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()        
        
        # 2. reproject from sinusoidal to epsg 4326 (resampling also should take place in this step, if necessary)
        command = [gdalwarp, '-t_srs','EPSG:4326','-tr', '0.01', '0.01', '-overwrite', out_file+'.tif', out_file+'_rep.tif']
        print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()                

        # 3. clip raster
        command = [gdal_translate, '-projwin', str(extent[0]),str(extent[3]),
                   str(extent[1]),str(extent[2]),'-of','GTiff', out_file+'_rep.tif', out_file+'_clip_b04.tif']
        print (sp.list2cmdline(command))
        norm = sp.Popen(sp.list2cmdline(command),stdout=sp.PIPE, shell=True)
        norm.communicate()        

"C:\Program Files\GDAL//gdal_translate.exe" -of GTiff HDF4_EOS:EOS_GRID:D:\Downloads\MOD09A1.A2008017.h27v05.006.2015169223429.hdf:MOD_Grid_500m_Surface_Reflectance:sur_refl_b04 D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//MOD09A1_2008-01-17.tif
"C:\Program Files\GDAL//gdalwarp.exe" -t_srs EPSG:4326 -tr 0.01 0.01 -overwrite D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//MOD09A1_2008-01-17.tif D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//MOD09A1_2008-01-17_rep.tif
"C:\Program Files\GDAL//gdal_translate.exe" -projwin 116.84693268 34.10324112 117.44 33.26 -of GTiff D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//MOD09A1_2008-01-17_rep.tif D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod09a1//MOD09A1_2008-01-17_clip_b04.tif


In [ ]:
# rename from iso date to date + dayofyear
files = listall(r'D:\Data\NDAI_VHI_GROUNDTRUTH\tif\mod13a2_month10_06', extension='.tif')
for i in files[1::]:
    try:
        print i
        year = int(i[-19:-15])
        month = int(i[-14:-12])
        day = int(i[-11:-9])
        date = datetime(year, month, day)
        date = np.datetime64(date)
        date = pd.Timestamp(np.datetime_as_string(date))    
        os.rename(i, i[:-19] + str(date.year) +'_'+ str(date.dayofyear).zfill(3) +'.tif')
    except: 
        continue